# Titanic

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [63]:
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

In [64]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Priprava podatkov

In [65]:
train = train[np.invert(np.isnan(train["Age"]))]
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
#test = test[np.invert(np.isnan(test["Age"]))]
#test = test[np.invert(np.isnan(test["Fare"]))]

Nov atribut minor:

In [66]:
def minors(x):
    minor = []
    for a in x["Age"]:
        if a < 16:
            minor = np.append(minor, 1)
        else:
            minor = np.append(minor, 0)
    x["Minor"] = minor

In [67]:
minors(train)
minors(test)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Nova atributa na podlagi priimkov:
* Vsaj en prezivel z istim priimkom
* Vsi umrli z istim priimkom

In [68]:
def allDied(x, y, ista):
    allD = []
    lived = []
    for index, row in x.iterrows():
        surname = row["Name"].split(",")[0]
        dead = 0
        alive = 0
        for index2, row2 in y.iterrows():
            if ista:
                if index != index2:
                    surname2 = row2["Name"].split(",")[0]
                    if surname2 == surname:
                        if row2["Survived"] == 1:
                            alive += 1
                        else:
                            dead += 1
            else:
                surname2 = row2["Name"].split(",")[0]
                if surname2 == surname:
                    if row2["Survived"] == 1:
                        alive += 1
                    else:
                        dead += 1
        if dead + alive < 2:
            allD = np.append(allD, 0)
            lived = np.append(lived, 0)
        else:
            if alive == 0:
                allD = np.append(allD, 1)
            else:
                allD = np.append(allD, 0)
            if alive >= 1:
                lived = np.append(lived, 1)
            else:
                lived = np.append(lived, 0)
    x["SurnameAllDied"] = allD
    x["SurnameOneLived"] = lived

In [69]:
allDied(train, train, True)
allDied(test, train, False)

In [70]:
train[train["SurnameOneLived"] == 1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Minor,SurnameAllDied,SurnameOneLived
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0.0,0.0,1.0
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,0.0,0.0,1.0
25,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,NaN,S,0.0,0.0,1.0
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S,0.0,0.0,1.0
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.0,1,2,SC/Paris 2123,41.5792,NaN,C,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,851,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,NaN,S,1.0,0.0,1.0
854,855,0,2,"Carter, Mrs. Ernest Courtenay (Lilian Hughes)",female,44.0,1,0,244252,26.0000,NaN,S,0.0,0.0,1.0
858,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,NaN,C,0.0,0.0,1.0
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S,1.0,0.0,1.0


Tabele za napovedovanje:

In [71]:
y = train["Survived"]
x = train[["Pclass", "Age","Fare", "Minor", "SurnameAllDied", "SurnameOneLived"]]
x["FamilySize"] = train["Parch"] + train["SibSp"]

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [72]:
def one_hot(name, x, y):
    category = np.array([])
    for a in y[name]:
        if a not in category:
            category = np.append(category, a)
    category = np.sort(category)
    for c in category:
        current = []
        for a in y[name]:
            if a == c:
                current = np.append(current, 1)
            else:
                current = np.append(current, 0)
        x[c] = current

In [73]:
one_hot("Sex", x, train)
#one_hot("Embarked", x, train)
x = x.loc[:, x.columns != 'nan']
#enc = OneHotEncoder()
#enc.fit(train)

/tmp/ipykernel_4056/1030216102.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [74]:
xtest = test[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived"]]
xtest["FamilySize"] = test["Parch"] + test["SibSp"]
one_hot("Sex", xtest, test)
#one_hot("Embarked", xtest, test)

/tmp/ipykernel_4056/1030216102.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [75]:
x

,Pclass,Age,Fare,Minor,SurnameAllDied,SurnameOneLived,FamilySize,female,male
0,3,22.0,7.2500,0.0,0.0,0.0,1,0.0,1.0
1,1,38.0,71.2833,0.0,0.0,0.0,1,1.0,0.0
2,3,26.0,7.9250,0.0,0.0,0.0,0,1.0,0.0
3,1,35.0,53.1000,0.0,0.0,0.0,1,1.0,0.0
4,3,35.0,8.0500,0.0,0.0,0.0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
885,3,39.0,29.1250,0.0,1.0,0.0,5,1.0,0.0
886,2,27.0,13.0000,0.0,0.0,0.0,0,0.0,1.0
887,1,19.0,30.0000,0.0,0.0,1.0,0,1.0,0.0
889,1,26.0,30.0000,0.0,0.0,0.0,0,0.0,1.0


## Napovedovanje

### Random forest

In [76]:
clf = RandomForestClassifier()
clf.fit(x,y)

RandomForestClassifier()

In [77]:
r = clf.predict(xtest)

In [78]:
resoult = test[["PassengerId"]]

In [79]:
resoult["Survived"] = r

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [80]:
resoult.to_csv("Predictions/rfc2.csv",  index=False)

### Naive Bayes

In [81]:
gnb = GaussianNB()
gnb.fit(x,y)
tt = gnb.predict(xtest)

In [82]:
resoult2 = test[["PassengerId"]]
resoult2["Survived"] = tt

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [83]:
resoult2 == resoult

,PassengerId,Survived
0,True,True
1,True,False
2,True,True
3,True,False
4,True,True
...,...,...
413,True,True
414,True,True
415,True,True
416,True,True


In [84]:
resoult2.to_csv("Predictions/nbc2.csv",  index=False)